# Embeddings

Привет! Сегодня ты поработаешь с эмбеддингами: сделаешь классификатор эмоции твитов. Для начала, загрузи их:

In [ ]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
84.9MB [00:00, 119MB/s] 
Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Заимпортируй библиотеки и сделай работу скриптов вопсроизводимой.

In [ ]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [ ]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Текст очень грязные. Надо добавить очистку текста в его предобработку. 

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Стокенизируем текст, избавим от знаков пунктуации и мелких слов.

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data . 

In [ ]:
word2vec = api.load("word2vec-google-news-300")

[=================================================-] 99.6% 1656.0/1662.8MB downloaded

In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Эмбеддинги не нормализированны, поэтому это тоже надо делать (нейросети и не только любят нормальные данные).

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        #print(tokens)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        #tokens_non_filtered = [self.tokenizer.tokenize(line.lower()) for line in text]
        tokens_non_filtered = self.tokenizer.tokenize(text.lower())
        #фильтруем
        tokens = [w for w in tokens_non_filtered if all(c not in string.punctuation for c in w) and len(w) > 2]

        return tokens

    def get_embeddings_(self, tokens):
        #embeddings =[(self.word2vec.get_vector(w) - self.mean) / self.std for w in line for line in tokens if w in word2vec and len(w) > 3]
        embeddings =[(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in word2vec]
       
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

## Average embedding
---
Попробуем получить векторное представление предложения из эмбеддингов слов. Самый простой вариант: усреднить вектора по всем словам. Полученный вектор можно отправить любому классификатору как вектор признаков.

Посмотрим, насколько хорошо усреднее работает для определение эмоций твитов. Сделаем их визуализацию.

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Можно получить более аккуратными алгоритмами, но данный алгоритм покажет сложность задачи и поможет оценить требования к классификатору.

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)

pca.fit(examples["features"])
examples["transformed_features"] = pca.transform(examples["features"]) 

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1002', ...)

Скорее всего, ты увидел не сильно различающиеся классы. Возможно, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую(хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Для обученния и тестирования нейросетевой модели сделаем отдельные функции.

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics.classification import accuracy_score

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device).float().unsqueeze(1)

        # Получи предсказания модели
        optimizer.zero_grad()
        pred = model(features)

        loss = criterion(pred, targets)

        # Обнови параметры модели
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].cpu().unsqueeze(1).float()
            #print(features)
            #print(targets)
            # Получи предсказания модели
            preds = model(features).detach().cpu()
            #print(preds)
            #print(preds[1])
            #print(preds.max(axis=1))
            loss = criterion(preds, targets) # Посчитай лосс
            acc = accuracy_score(targets.numpy(), (torch.sigmoid(preds)).numpy()>0.5) # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}
    #return features, targets, preds

Создадим модель, оптимизатор и целевую функцию. Можете создавать любую модель, с любым количеством слоев, функций активации и прочее.


In [ ]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 3
hidden = 20
print(vector_size)
model = nn.Sequential(
    nn.Linear(vector_size, hidden ),
    nn.BatchNorm1d(hidden),
    nn.ReLU(),
    nn.Linear(hidden, hidden),
    nn.BatchNorm1d(hidden),
    nn.ReLU(),
    nn.Linear(hidden,1),
    #nn.BatchNorm1d(hidden),
    #nn.ReLU(),
    #nn.Linear(hidden,1)

)
model = model.cuda()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

200


Обучим модель и протестируем её.
После каждой эпохи, проверим качество модели по валидационной части датасета. Если метрика стала лучше, будем сохранять модель. Подумай, какая метрика (точность или лосс) будет лучше работать в этой задаче? 

Здесь и далее реализованно с помощью лосс. Если думаешь, что лучше сравнивать модель через качество, то поменяй код выбора модели.

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.48028661489486696, 'Test Acc': 0.7676484375}



{'Test Loss': 0.4750184272527695, 'Test Acc': 0.7715625}



{'Test Loss': 0.4736748864650726, 'Test Acc': 0.77169140625}


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.4754124887454243, 'Test Acc': 0.7702706918929713}


## TF-iDF
---

Вместо обычного усреднения эмбеддингов их можно дополнительно перевзвесить. Для этого воспользуемся алгоритмом `TD-iDF`. Он уже реализован в библиотеке `scikit-learn`, остается только его добавить в наш пайплайн.

In [ ]:
from collections import defaultdict
from typing import Dict

from sklearn.feature_extraction.text import TfidfVectorizer


class TwitterDatasetTfIdf(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, weights: Dict[str, float] = None):
        super().__init__(data, feature_column, target_column, word2vec)

        if weights is None:
            self.weights = self.get_tf_idf_()
        else:
            self.weights = weights
   
        

        

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std  * self.weights.get(token, 1) for token in tokens if token in word2vec]
        #embeddings =[(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in word2vec]
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_tf_idf_(self):

        text = self.data[self.feature_column]

        tokens_non_filtered = [self.tokenizer.tokenize(line.lower()) for line in text]

        tokens = [w for w in tokens_non_filtered if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        

        # Надо обучить tfidf на очищенном тексте. Но он принимает только список текстов, а не список списка токенов. Надо превратить второе в первое
        tokenized_texts = [' '.join(line) for line in tokens]
        tf_idf = TfidfVectorizer()
        # Обучи tf-idf
        tf_idf.fit(tokenized_texts)
        return dict(zip(tf_idf.get_feature_names(), tf_idf.idf_))
       


In [ ]:
dev = TwitterDatasetTfIdf(dev_data, "text", "emotion", word2vec)

Посмотрим на сложность получившейся задачи используя визуализацию через `PCA`.

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

pca.fit(examples["features"])
examples["transformed_features"] = pca.transform(examples["features"]) 

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1108', ...)

Создать нейросетку, обучим её на этих данных.

In [ ]:
train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
# Не забудь поиграться с параметрами ;)

vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 3

model = nn.Sequential(
    nn.Linear(vector_size, hidden ),
    nn.BatchNorm1d(hidden),
    nn.ReLU(),
    nn.Linear(hidden, hidden),
    nn.BatchNorm1d(hidden),
    nn.ReLU(),
    nn.Linear(hidden,1)
)
model = model.cuda()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    print(testing(model, criterion, valid_loader, device))
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model2.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5007837518453598, 'Test Acc': 0.7530703125}
{'Test Loss': 0.4736748864650726, 'Test Acc': 0.77169140625}



{'Test Loss': 0.4951369457244873, 'Test Acc': 0.7569296875}
{'Test Loss': 0.4736748864650726, 'Test Acc': 0.77169140625}



{'Test Loss': 0.4933911567926407, 'Test Acc': 0.75768359375}
{'Test Loss': 0.4736748864650726, 'Test Acc': 0.77169140625}


In [ ]:
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec, weights=dev.weights)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model2.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.4988749072955439, 'Test Acc': 0.7549701727236422}


Есть ли разница в качестве между способами? Получилось ли улучшить качество модели?

----
При прочих равных параметрах, точность с TF-IDF получается меньше, на ~ 2%

----

Сделай небольшое исследование:
- Попробуй сделать несколько нейросеток в качестве классификатора
- Попробуй другие предобученные эмбеддинги
- Попробуй очистить текст от ников ("@username"), url-ов и других символов

Для реализации последнего тебе могут помочь регулярные выражения (`import re`). Напише ниже отчет, что ты попробовал и что получилось.

---

1.   Производил обучние с разными праметрами модели (кол-во слоев, лернинг рейт, метрики) - большого прироста или ухудшения получено не было.

2.   Также попробовал обучить модель с другим предобученным эмбеддингом (glove-twitter-200), результаты улучшились всего на ~ 0.5%

3. Очистка твитов от ссылок и юзернеймов, с помощью регулярных выражений, так же не дала сильного прироста точности. Это можно объяснить тем, что токенизация и фильтрация и так убирает практически все юзернеймы с сыллками.

4. Самый большой прирост точности (3%), был получен при использовании токенов от 2 символов (вместо от 3 как было изначально в ноутбуке)

5. Максимальная итоговая точность составила 77.8% в моделе без TF-IDF, с очисткой от юзернеймов и ссылок, и с предобученным эмбеддингом glove-twitter-200

---

In [ ]:
#Воспользуемся другим предобученным эмбеддингом (glove-twitter-200), что бы не менять тело функций, назовем его так же.
word2vec = api.load("glove-twitter-200")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#Результаты полученные с (glove-twitter-200)
#Без TF-IDF
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.4693148683309555, 'Test Acc': 0.77310546875}



{'Test Loss': 0.4719934871196747, 'Test Acc': 0.77116015625}



{'Test Loss': 0.46234872937202454, 'Test Acc': 0.7773671875}


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.46227069080066374, 'Test Acc': 0.7779615115814696}


In [ ]:
#С TF-IDF
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    print(testing(model, criterion, valid_loader, device))
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model2.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.4822570389509201, 'Test Acc': 0.7638359375}
{'Test Loss': 0.46234872937202454, 'Test Acc': 0.7773671875}



{'Test Loss': 0.4799374483823776, 'Test Acc': 0.7659296875}
{'Test Loss': 0.46234872937202454, 'Test Acc': 0.7773671875}



{'Test Loss': 0.47667491233348847, 'Test Acc': 0.7680078125}
{'Test Loss': 0.46234872937202454, 'Test Acc': 0.7773671875}


In [ ]:
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec, weights=dev.weights)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model2.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.47985114923681316, 'Test Acc': 0.7670539636581469}


In [ ]:
# предварительно очистим текст от юзернеймов и ссылок

In [ ]:
from torch.utils.data import Dataset, random_split
import re

class TwitterDataset2(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
       
        filtered = re.sub(r'htt\S+', '', self.data[self.feature_column][item])
        text = re.sub(r'@\S+', '', filtered)

        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)

        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}
        

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens_non_filtered = self.tokenizer.tokenize(text.lower())
        #фильтруем
        tokens = [w for w in tokens_non_filtered if all(c not in string.punctuation for c in w) and len(w) > 2]

        return tokens

    def get_embeddings_(self, tokens):
        #embeddings =[(self.word2vec.get_vector(w) - self.mean) / self.std for w in line for line in tokens if w in word2vec and len(w) > 3]
        embeddings =[(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in word2vec]
       
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset2(dev_data, "text", "emotion", word2vec)

In [ ]:
#повторим опыты 
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
Traceback (most recent call last):
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
    sel

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
    self._shutdown_workers()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1075, in _shutd


{'Test Loss': 0.4643804495334625, 'Test Acc': 0.77657421875}


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
Traceback (most recent call last):
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
  F

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
Traceback (most recent call last):
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f911ef2a208>>
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
Traceback (most recent call last):
    self._shutdown_workers()
  


{'Test Loss': 0.46560154771804807, 'Test Acc': 0.775421875}



{'Test Loss': 0.4613413701057434, 'Test Acc': 0.7781875}


In [ ]:
test = TwitterDataset(test_data, "text", "emotion", word2vec)

In [ ]:
test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.4653361005524096, 'Test Acc': 0.7756464656549521}


In [ ]:
from collections import defaultdict
from typing import Dict

from sklearn.feature_extraction.text import TfidfVectorizer


class TwitterDatasetTfIdf2(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, weights: Dict[str, float] = None):
        super().__init__(data, feature_column, target_column, word2vec)

        if weights is None:
            self.weights = self.get_tf_idf_()
        else:
            self.weights = weights
   
        

        

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std  * self.weights.get(token, 1) for token in tokens if token in word2vec]
        #embeddings =[(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in word2vec]
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_tf_idf_(self):

        text = self.data[self.feature_column]

        #Фильтрация текста от юзернемов и ссылок
        filtered_text = [re.sub(r'htt\S+', '', line) for line in text]
        filtered_text = [re.sub(r'@\S+', '', line) for line in filtered_text]

        tokens_non_filtered = [self.tokenizer.tokenize(line.lower()) for line in filtered_text]

        tokens = [w for w in tokens_non_filtered if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        

        # Надо обучить tfidf на очищенном тексте. Но он принимает только список текстов, а не список списка токенов. Надо превратить второе в первое
        tokenized_texts = [' '.join(line) for line in tokens]
        tf_idf = TfidfVectorizer()
        # Обучи tf-idf
        tf_idf.fit(tokenized_texts)
        return dict(zip(tf_idf.get_feature_names(), tf_idf.idf_))

In [ ]:
dev = TwitterDatasetTfIdf2(dev_data, "text", "emotion", word2vec)

In [ ]:
#С TF-IDF
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    print(testing(model, criterion, valid_loader, device))
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model2.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.4957008328437805, 'Test Acc': 0.7549921875}
{'Test Loss': 0.4613413701057434, 'Test Acc': 0.7781875}



{'Test Loss': 0.48109661173820495, 'Test Acc': 0.7650859375}
{'Test Loss': 0.4613413701057434, 'Test Acc': 0.7781875}



{'Test Loss': 0.4782911947965622, 'Test Acc': 0.76737109375}
{'Test Loss': 0.4613413701057434, 'Test Acc': 0.7781875}


In [ ]:
test = TwitterDatasetTfIdf2(test_data, "text", "emotion", word2vec, weights=dev.weights)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model2.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.49334974810719107, 'Test Acc': 0.7572633785942492}
